In [2]:
# Used Car Price Prediction - Data Preprocessing
# Google Colab Solution (without Google Drive)

# Import libraries
import pandas as pd
import numpy as np
from datetime import datetime
from IPython.display import display

# Section 1: File Upload
print("=== STEP 1: Upload train.csv ===")
print("Please upload the train.csv file when prompted")
from google.colab import files
uploaded = files.upload()

# Verify upload
if 'train.csv' not in uploaded:
    raise FileNotFoundError("Please upload train.csv file")

# Load the dataset
df = pd.read_csv('train.csv')
print("\nFile successfully loaded!")
print("Initial shape:", df.shape)

=== STEP 1: Upload train.csv ===
Please upload the train.csv file when prompted


Saving train.csv to train.csv

File successfully loaded!
Initial shape: (5847, 14)


a) Handle Missing Values

In [3]:
# a) Handle missing values
print("\n=== PART a) Handling Missing Values ===")

# Display initial missing values
print("\nMissing values before handling:")
display(df.isnull().sum())

# Handle each column with missing values:
# 1. New_Price - Many missing values, not critical for prediction
df = df.drop('New_Price', axis=1)
print("\n✔ Dropped 'New_Price' column (many missing values)")

# 2. Mileage - Extract numbers first, then impute
df['Mileage'] = df['Mileage'].str.extract('(\d+\.?\d*)')[0].astype(float)
df['Mileage'] = df.groupby('Fuel_Type')['Mileage'].transform(lambda x: x.fillna(x.median()))
print("✔ Imputed Mileage with median by Fuel_Type")

# 3. Engine - Extract CC values, then impute
df['Engine'] = df['Engine'].str.extract('(\d+)')[0].astype(float)
df['Engine'] = df.groupby(['Name', 'Fuel_Type'])['Engine'].transform(lambda x: x.fillna(x.median()))
print("✔ Imputed Engine with median by Name and Fuel_Type")

# 4. Power - Extract bhp values, then impute
df['Power'] = df['Power'].str.extract('(\d+\.?\d*)')[0].astype(float)
df['Power'] = df.groupby(['Name', 'Fuel_Type'])['Power'].transform(lambda x: x.fillna(x.median()))
print("✔ Imputed Power with median by Name and Fuel_Type")

# 5. Seats - Impute with mode
df['Seats'] = df['Seats'].fillna(df['Seats'].mode()[0])
print("✔ Imputed Seats with mode")

# Verify missing values after handling
print("\nMissing values after handling:")
display(df.isnull().sum())

print("\nJustification:")
print("- Dropped 'New_Price' (too many missing, not critical)")
print("- Used group medians for technical specs (maintains relationships)")
print("- Used mode for 'Seats' (categorical-like feature)")


=== PART a) Handling Missing Values ===

Missing values before handling:


,0
Unnamed: 0,0
Name,0
Location,0
Year,0
Kilometers_Driven,0
Fuel_Type,0
Transmission,0
Owner_Type,0
Mileage,2
Engine,36



✔ Dropped 'New_Price' column (many missing values)
✔ Imputed Mileage with median by Fuel_Type
✔ Imputed Engine with median by Name and Fuel_Type
✔ Imputed Power with median by Name and Fuel_Type
✔ Imputed Seats with mode

Missing values after handling:


,0
Unnamed: 0,0
Name,0
Location,0
Year,0
Kilometers_Driven,0
Fuel_Type,0
Transmission,0
Owner_Type,0
Mileage,2
Engine,31



Justification:
- Dropped 'New_Price' (too many missing, not critical)
- Used group medians for technical specs (maintains relationships)
- Used mode for 'Seats' (categorical-like feature)


b) Remove Units from Columns

In [4]:
# b) Remove units from columns
print("\n=== PART b) Removing Units from Columns ===")

# Already handled during missing value treatment, but verify
print("\nSample values after unit removal:")
print("Mileage:", df['Mileage'].head().values)
print("Engine:", df['Engine'].head().values)
print("Power:", df['Power'].head().values)

print("\n✔ Units removed from:")
print("- Mileage (removed 'kmpl'/'km/kg')")
print("- Engine (removed 'CC')")
print("- Power (removed 'bhp')")


=== PART b) Removing Units from Columns ===

Sample values after unit removal:
Mileage: [19.67 13.   20.77 15.2  23.08]
Engine: [1582. 1199. 1248. 1968. 1461.]
Power: [126.2   88.7   88.76 140.8   63.1 ]

✔ Units removed from:
- Mileage (removed 'kmpl'/'km/kg')
- Engine (removed 'CC')
- Power (removed 'bhp')


c) One-Hot Encode Categorical Variables

In [5]:
# c) One-hot encode categorical variables
print("\n=== PART c) One-Hot Encoding ===")

print("\nOriginal categorical values:")
print("Fuel_Type:", df['Fuel_Type'].unique())
print("Transmission:", df['Transmission'].unique())
print("Owner_Type:", df['Owner_Type'].unique())

# Perform one-hot encoding
df = pd.get_dummies(df, columns=['Fuel_Type', 'Transmission', 'Owner_Type'], drop_first=True)

print("\n✔ One-hot encoding complete")
print("New binary columns created:")
print([col for col in df.columns if col.startswith(('Fuel_Type', 'Transmission', 'Owner_Type'))][:5], "...")


=== PART c) One-Hot Encoding ===

Original categorical values:
Fuel_Type: ['Diesel' 'Petrol' 'Electric']
Transmission: ['Manual' 'Automatic']
Owner_Type: ['First' 'Second' 'Fourth & Above' 'Third']

✔ One-hot encoding complete
New binary columns created:
['Fuel_Type_Electric', 'Fuel_Type_Petrol', 'Transmission_Manual', 'Owner_Type_Fourth & Above', 'Owner_Type_Second'] ...


d) Create New Feature

In [6]:
# d) Create new feature
print("\n=== PART d) Creating New Feature ===")

# Create Car_Age feature
current_year = datetime.now().year
df['Car_Age'] = current_year - df['Year']

print("\n✔ Added 'Car_Age' feature (Current Year - Manufacturing Year)")
print("\nCar age distribution:")
print(df['Car_Age'].describe())

display(df[['Name', 'Year', 'Car_Age']].head())


=== PART d) Creating New Feature ===

✔ Added 'Car_Age' feature (Current Year - Manufacturing Year)

Car age distribution:
count    5847.000000
mean       11.551565
std         3.194949
min         6.000000
25%         9.000000
50%        11.000000
75%        13.000000
max        27.000000
Name: Car_Age, dtype: float64


,Name,Year,Car_Age
0,Hyundai Creta 1.6 CRDi SX Option,2015,10
1,Honda Jazz V,2011,14
2,Maruti Ertiga VDI,2012,13
3,Audi A4 New 2.0 TDI Multitronic,2013,12
4,Nissan Micra Diesel XV,2013,12


e) Data Manipulation Operations

In [7]:
# e) Data manipulation operations
print("\n=== PART e) Data Operations ===")

# Create dictionary to store all operation results
results = {}

# 1. Select specific columns
results['selected'] = df[['Name', 'Location', 'Year', 'Car_Age', 'Price']]
print("\n1. SELECT: Name, Location, Year, Car_Age, Price")
display(results['selected'].head())

# 2. Filter expensive cars (Price > 20 lakhs)
results['filtered'] = df[df['Price'] > 20]
print("\n2. FILTER: Price > 20 lakhs")
print(f"Found {len(results['filtered'])} cars")
display(results['filtered'][['Name', 'Price']].head())

# 3. Rename columns
results['renamed'] = df.rename(columns={
    'Kilometers_Driven': 'Kms_Driven',
    'Engine': 'Engine_CC',
    'Power': 'Power_bhp'
})
print("\n3. RENAME: Kilometers_Driven→Kms_Driven, Engine→Engine_CC, Power→Power_bhp")
display(results['renamed'][['Kms_Driven', 'Engine_CC', 'Power_bhp']].head())

# 4. Mutate - create Price_Per_Km
results['mutated'] = df.copy()
results['mutated']['Price_Per_Km'] = results['mutated']['Price'] / results['mutated']['Kilometers_Driven']
print("\n4. MUTATE: Added Price_Per_Km (Price/Kms_Driven)")
display(results['mutated'][['Name', 'Price', 'Kilometers_Driven', 'Price_Per_Km']].head())

# 5. Arrange by Price descending
results['arranged'] = df.sort_values('Price', ascending=False)
print("\n5. ARRANGE: By Price (descending)")
display(results['arranged'][['Name', 'Price']].head())

# 6. Summarize with groupby
if 'Fuel_Type_Diesel' in df.columns:
    results['summary'] = df.groupby(['Location', 'Fuel_Type_Diesel'])['Price'].agg(['mean', 'count'])
    print("\n6. SUMMARIZE: Average price by Location and Fuel Type")
    display(results['summary'].head())
else:
    print("\n6. Could not create summary - Fuel_Type columns missing")


=== PART e) Data Operations ===

1. SELECT: Name, Location, Year, Car_Age, Price


,Name,Location,Year,Car_Age,Price
0,Hyundai Creta 1.6 CRDi SX Option,Pune,2015,10,12.50
1,Honda Jazz V,Chennai,2011,14,4.50
2,Maruti Ertiga VDI,Chennai,2012,13,6.00
3,Audi A4 New 2.0 TDI Multitronic,Coimbatore,2013,12,17.74
4,Nissan Micra Diesel XV,Jaipur,2013,12,3.50



2. FILTER: Price > 20 lakhs
Found 687 cars


,Name,Price
11,Land Rover Range Rover 2.2L Pure,27.00
17,Mercedes-Benz New C-Class C 220 CDI BE Avantgare,28.00
20,Audi A6 2011-2015 35 TFSI Technology,23.50
36,Mercedes-Benz M-Class ML 350 CDI,28.00
48,Audi A4 2.0 TDI 177 Bhp Premium Plus,21.43



3. RENAME: Kilometers_Driven→Kms_Driven, Engine→Engine_CC, Power→Power_bhp


...


4. MUTATE: Added Price_Per_Km (Price/Kms_Driven)


,Name,Price,Kilometers_Driven,Price_Per_Km
0,Hyundai Creta 1.6 CRDi SX Option,12.50,41000,0.000305
1,Honda Jazz V,4.50,46000,0.000098
2,Maruti Ertiga VDI,6.00,87000,0.000069
3,Audi A4 New 2.0 TDI Multitronic,17.74,40670,0.000436
4,Nissan Micra Diesel XV,3.50,86999,0.000040



5. ARRANGE: By Price (descending)


,Name,Price
3952,Land Rover Range Rover 3.0 Diesel LWB Vogue,160.00
5620,Lamborghini Gallardo Coupe,120.00
5752,Jaguar F Type 5.0 V8 S,100.00
1457,Land Rover Range Rover Sport SE,97.07
1917,BMW 7 Series 740Li,93.67



6. Could not create summary - Fuel_Type columns missing


In [8]:
# Save and download results
print("\n=== Saving Results ===")

# Save processed dataframe
df.to_csv('processed_used_cars.csv', index=False)
print("✔ Saved processed data to 'processed_used_cars.csv'")

# Save all operation results
for name, result in results.items():
    result.to_csv(f'{name}_result.csv', index=False)
    print(f"✔ Saved {name} operation to '{name}_result.csv'")

# Create zip file of all results
!zip all_results.zip *.csv

# Download all files
print("\nDownloading all files...")
files.download('all_results.zip')
print("✔ Download complete! Check your downloads folder")


=== Saving Results ===
✔ Saved processed data to 'processed_used_cars.csv'
✔ Saved selected operation to 'selected_result.csv'
✔ Saved filtered operation to 'filtered_result.csv'
✔ Saved renamed operation to 'renamed_result.csv'
✔ Saved mutated operation to 'mutated_result.csv'
✔ Saved arranged operation to 'arranged_result.csv'
  adding: arranged_result.csv (deflated 82%)
  adding: filtered_result.csv (deflated 81%)
  adding: mutated_result.csv (deflated 75%)
  adding: processed_used_cars.csv (deflated 79%)
  adding: renamed_result.csv (deflated 79%)
  adding: selected_result.csv (deflated 80%)
  adding: train.csv (deflated 79%)



<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

✔ Download complete! Check your downloads folder
